## Import Packages

In [7]:
import os
import json
import datetime
import urllib.error
import urllib.parse
import urllib.request

import pytz
import ipywidgets
import ipydatetime

import matplotlib.pyplot as plt
import pandas as pd

from functools import reduce

# from here_map_widget import Map, GeoJSON, Marker
from ipyleaflet import Map, GeoJSON, Marker, TileLayer
from ipyleaflet.external.here import basemap_to_tiles, basemaps

import folium
from folium import plugins

from decimal import Decimal, ROUND_HALF_UP, ROUND_DOWN

# pd.set_option('max_rows', 10)
pd.set_option('max_rows', None)

## Define Functions

In [8]:
def tracking_login(email, password):
    url = 'https://tracking.api.here.com/users/v2/login'
    headers = {'Content-Type': 'application/json'}
    body = dict(email=email, password=password)
    data = json.dumps(body).encode()
    print(url)
    req = urllib.request.Request(url, headers=headers, data=data, method='POST')
    with urllib.request.urlopen(req) as res:
        data = res.read()
        return json.loads(data).get('accessToken')
    
    
def tracking_get_device_list(access_token, **kwargs):
    base_url = f'https://tracking.api.here.com/users/v2/devices'
    params = urllib.parse.urlencode(dict(**kwargs))
    url = f'{base_url}?{params}'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }
    print(url)
    req = urllib.request.Request(url, headers=headers)
    with urllib.request.urlopen(req) as res:
        data = res.read()
        return json.loads(data).get('data')

    
def tracking_get_shadow(access_token, tracking_id, **kwargs):
    base_url = f'https://tracking.api.here.com/shadows/v2/{tracking_id}'
    params = urllib.parse.urlencode(dict(**kwargs))
    url = f'{base_url}?{params}'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }
    print(url)
    req = urllib.request.Request(url, headers=headers)
    with urllib.request.urlopen(req) as res:
        data = res.read()
        return json.loads(data)
    

def tracking_get_trace(access_token, tracking_id, **kwargs):
    base_url = f'https://tracking.api.here.com/traces/v2/{tracking_id}'
    params = urllib.parse.urlencode(dict(**kwargs))
    url = f'{base_url}?{params}'
    headers = {
        'Authorization': f'Bearer {access_token}',
        'Content-Type': 'application/json',
    }
    print(url)
    req = urllib.request.Request(url, headers=headers)
    with urllib.request.urlopen(req) as res:
        data = res.read()
        return json.loads(data)

In [9]:
def match_route(apikey, mode, trace, **kwargs):
    base_url = 'https://fleet.ls.hereapi.com/2'
    resource_uri = 'calculateroute.json'
    headers = {'Content-Type': 'text/plain'}
    params = urllib.parse.urlencode(dict(routeMatch=1, apiKey=apikey, mode=mode, **kwargs))
    data = trace.encode()
    url = f'{base_url}/{resource_uri}?{params}'
    print(url)
    req = urllib.request.Request(url, headers=headers, data=data, method='POST')
    try:
        with urllib.request.urlopen(req) as res:
            data = res.read()
            return json.loads(data)
    except urllib.error.HTTPError as e:
        print(e, e.reason, e.code, e.read())

## Define Variables

In [10]:
TRACKING_EMAIL = 'xxx'
TRACKING_PASSWORD = 'xxx'
APIKEY = 'xxx'

project_id = 'xxx'
device_id = 'xxx'

## User Login

In [11]:
access_token = tracking_login(TRACKING_EMAIL, TRACKING_PASSWORD)
access_token

https://tracking.api.here.com/users/v2/login


'h1.hPdNtW48uiT0ZZPswKoT8Q.nu1YbCljOuT-0lpXieAKv--u5rMou383j7AGCgGcDEz7p822AKBRj0zxtx9L-vOgrR9nQfiM6ebvVQPssNWqyEdiukg0gdfIkeusO1Z-Y0QKG67laXsK5ig1oxO4pZGmQJZ8QP9-kHd4Pa4oJEFVNFOrzqgFPLHMHK19jaYn_aGpHPffVBNpFSX68UsuExUt0s5VCWDLIsnmt82qncE1gRn66r8TcAU04z2JazmgsZsHjVeXuXvsc_Y4pMgAbjnWYZwvWudEdw-LRQ1hIcafeWtxDQwthFZzErLOQUkow9ybDlL7_QusgHkO24JYO1ciozaHa7_SHuAPTGDVd3DDTXgG6jOHxy_UwVOEN2V_Qyjxbh63vkOT6Wc24Uh-XubOQwK10e8zdDnYUGqwqBVa4He6GsBDRairEji4sefwoM-_pFFiqU-jUKbMm96BkJakUrohriBmOssUlc8b7jDznw.TNPr8v1UW1kDaJH7LwpH7i71OOHGvSSCdjTDxPP6qjChA57yJ2zXmgYLhWSpVxsk_xWnkP09Ph1yxXBvE9uPdKjLtZo5F-zVjkX7XRoqSo5huFk8X4PEbUmPXmAvsaK-8FJfzO7sm-5rn6LlSnx_G8jhrRttbxZPl7Mjh61NuqGMRTxQVJIPYEHmEQmla105rGwv6I4JB2Rbw9yFFMo_x2rpCJM7CasM2k5_xl9k9YDqEeloC5XKkaclMXf-DydlBqIMdV42cD2S6LaOwAdOg4zJwos9nlXbmxyEZerb_G3xPYSE-LVU-rJekTAvJeEO7Oy61KXHNu9fMuuNaFzvnA'

## Get user's list of devices

In [12]:
device_list = tracking_get_device_list(access_token, projectId=project_id)
device_list

https://tracking.api.here.com/users/v2/devices?projectId=CUST-7d3d712c-e368-40a3-892d-b572d4491874


[{'shadow': {'desired': {'payload': {'refAppData': {'icon': 'GENERIC',
      'name': '田邊私用端末'}},
    'system': {'stateVersion': 5,
     'rate': {'sendMs': 300000, 'sampleMs': 60000, 'distanceM': 0},
     'lastModifiedGeofenceTimestamp': 0},
    'timestamp': 1610955861383},
   'reported': {'payload': {'refAppData': {'icon': 'GENERIC',
      'name': '田邊私用端末',
      'nwpos': True}},
    'system': {'stateVersion': 77248,
     'client': {'name': 'HERE Tracker',
      'model': 'Pixel 3a',
      'version': '1.1.5',
      'firmware': 'RQ2A.210305.006',
      'platform': 'android',
      'manufacturer': 'Google'},
     'computed': {'moving': False,
      'lastMovedTimestamp': 1615505721620,
      'online': True},
     'reportedSensorData': {'batteryLevel': 80, 'batteryIsCharging': False}},
    'timestamp': 1615513740731,
    'position': {'alt': 145,
     'lat': 35.6799996,
     'lng': 139.764204,
     'type': 'cell',
     'accuracy': 2026,
     'altaccuracy': 710}}},
  'trackingId': 'HERE-230a4

In [13]:
pd.DataFrame(device_list)

,shadow,trackingId
0,{'desired': {'payload': {'refAppData': {'icon'...,HERE-230a4588-74b8-479e-b4ea-73344b2200d6
1,{'desired': {'payload': {'refAppData': {'icon'...,HERE-7c0af8ad-39cf-4c73-97ca-211fe03557f4
2,{'desired': {'payload': {'refAppData': {'icon'...,HERE-5c02456f-e379-4f55-ba77-095d7d9d8c74
3,{'desired': {'payload': {'refAppData': {'icon'...,HERE-01f24170-a342-4bb4-a434-e7333f7e134d
4,{'desired': {'payload': {'refAppData': {'icon'...,HERE-054741ff-3318-4dda-b609-e4fe68c2f03e
5,{'desired': {'payload': {'refAppData': {'icon'...,HERE-89b7eb6c-be16-464a-a16c-ed0196fd82bd
6,{'desired': {'payload': {'refAppData': {'icon'...,HERE-d612a574-fec0-4ac1-abb1-c883ffb073b8


In [14]:
# 端末を選択
tracking_id = device_list[0].get('trackingId')
tracking_id

'HERE-230a4588-74b8-479e-b4ea-73344b2200d6'

## Retrieve the device shadow

In [15]:
shadow = tracking_get_shadow(access_token, tracking_id, projectId=project_id)
shadow

https://tracking.api.here.com/shadows/v2/HERE-230a4588-74b8-479e-b4ea-73344b2200d6?projectId=CUST-7d3d712c-e368-40a3-892d-b572d4491874


{'desired': {'payload': {'refAppData': {'icon': 'GENERIC', 'name': '田邊私用端末'}},
  'system': {'stateVersion': 5,
   'rate': {'sendMs': 300000, 'sampleMs': 60000, 'distanceM': 0},
   'lastModifiedGeofenceTimestamp': 0},
  'timestamp': 1610955861383},
 'reported': {'payload': {'refAppData': {'icon': 'GENERIC',
    'name': '田邊私用端末',
    'nwpos': True}},
  'system': {'stateVersion': 77248,
   'client': {'name': 'HERE Tracker',
    'model': 'Pixel 3a',
    'version': '1.1.5',
    'firmware': 'RQ2A.210305.006',
    'platform': 'android',
    'manufacturer': 'Google'},
   'computed': {'moving': False,
    'lastMovedTimestamp': 1615505721620,
    'online': True},
   'reportedSensorData': {'batteryLevel': 80, 'batteryIsCharging': False}},
  'timestamp': 1615513740731,
  'position': {'alt': 145,
   'lat': 35.6799996,
   'lng': 139.764204,
   'type': 'cell',
   'accuracy': 2026,
   'altaccuracy': 710}}}

## Retrive the device trace

In [16]:
jst = pytz.timezone('Asia/Tokyo')
after_picker = ipydatetime.DatetimePicker(description='After:', value=jst.localize(datetime.datetime(
    2021, 1, 19, 17, 30, 0
)))
before_picker = ipydatetime.DatetimePicker(description='Before:', value=jst.localize(datetime.datetime(
    2021, 1, 19,20 , 0, 0
)))
display(after_picker)
display(before_picker)

DatetimePicker(value=datetime.datetime(2021, 1, 19, 17, 30, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>), …

DatetimePicker(value=datetime.datetime(2021, 1, 19, 20, 0, tzinfo=<DstTzInfo 'Asia/Tokyo' JST+9:00:00 STD>), d…

In [17]:
after = int(after_picker.value.timestamp() * 1000)
before = int(before_picker.value.timestamp() * 1000)
trace = tracking_get_trace(access_token, tracking_id, projectId=project_id, after=after, before=before)
trace

https://tracking.api.here.com/traces/v2/HERE-230a4588-74b8-479e-b4ea-73344b2200d6?projectId=CUST-7d3d712c-e368-40a3-892d-b572d4491874&after=1611045000000&before=1611054000000


{'count': 149,
 'data': [{'timestamp': 1611053946054,
   'position': {'alt': 49,
    'lat': 35.33830382,
    'lng': 139.50076906,
    'type': 'gnss',
    'speed': 0,
    'accuracy': 16.150562286376953,
    'timestamp': 1611053956000,
    'altaccuracy': 1},
   'system': {'client': {'name': 'HERE Tracker',
     'model': 'Pixel 3a',
     'version': '1.1.5',
     'firmware': 'RQ1A.210105.002',
     'platform': 'android',
     'manufacturer': 'Google'},
    'reportedSensorData': {'batteryLevel': 37, 'batteryIsCharging': False}},
   'serverTimestamp': 1611054100975},
  {'timestamp': 1611053886049,
   'position': {'alt': 49,
    'lat': 35.33830283,
    'lng': 139.50076558,
    'type': 'gnss',
    'speed': 0,
    'accuracy': 6.694888114929199,
    'timestamp': 1611053896000,
    'altaccuracy': 1},
   'system': {'client': {'name': 'HERE Tracker',
     'model': 'Pixel 3a',
     'version': '1.1.5',
     'firmware': 'RQ1A.210105.002',
     'platform': 'android',
     'manufacturer': 'Google'},
   

In [18]:
pd.DataFrame(trace.get('data'))

,timestamp,position,system,serverTimestamp
0,1611053946054,"{'alt': 49, 'lat': 35.33830382, 'lng': 139.500...","{'client': {'name': 'HERE Tracker', 'model': '...",1611054100975
1,1611053886049,"{'alt': 49, 'lat': 35.33830283, 'lng': 139.500...","{'client': {'name': 'HERE Tracker', 'model': '...",1611054100975
2,1611053825947,"{'alt': 49, 'lat': 35.33829415, 'lng': 139.500...","{'client': {'name': 'HERE Tracker', 'model': '...",1611054100975
3,1611053765918,"{'alt': 52, 'lat': 35.33826498, 'lng': 139.500...","{'client': {'name': 'HERE Tracker', 'model': '...",1611053792125
4,1611053705895,"{'alt': 53, 'lat': 35.33825346, 'lng': 139.500...","{'client': {'name': 'HERE Tracker', 'model': '...",1611053792125
5,1611053645875,"{'alt': 52, 'lat': 35.33825408, 'lng': 139.500...","{'client': {'name': 'HERE Tracker', 'model': '...",1611053792125
6,1611053585771,"{'alt': 53, 'lat': 35.33824909, 'lng': 139.500...","{'client': {'name': 'HERE Tracker', 'model': '...",1611053792125
7,1611053525749,"{'alt': 52, 'lat': 35.33825934, 'lng': 139.500...","{'client': {'name': 'HERE Tracker', 'model': '...",1611053792125
8,1611053465660,"{'alt': 51, 'lat': 35.33826772, 'lng': 139.500...","{'client': {'name': 'HERE Tracker', 'model': '...",1611053492125
9,1611053405559,"{'alt': 51, 'lat': 35.3382739, 'lng': 139.5006...","{'client': {'name': 'HERE Tracker', 'model': '...",1611053492125


In [19]:
df = pd.json_normalize(trace.get('data'))
print(len(df))
df

149


,timestamp,serverTimestamp,position.alt,position.lat,position.lng,position.type,position.speed,position.accuracy,position.timestamp,position.altaccuracy,system.client.name,system.client.model,system.client.version,system.client.firmware,system.client.platform,system.client.manufacturer,system.reportedSensorData.batteryLevel,system.reportedSensorData.batteryIsCharging,position.heading
0,1611053946054,1611054100975,49.0,35.338304,139.500769,gnss,0.0,16.150562,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
1,1611053886049,1611054100975,49.0,35.338303,139.500766,gnss,0.0,6.694888,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
2,1611053825947,1611054100975,49.0,35.338294,139.500772,gnss,0.0,7.703820,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
3,1611053765918,1611053792125,52.0,35.338265,139.500737,gnss,0.0,10.166524,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
4,1611053705895,1611053792125,53.0,35.338253,139.500722,gnss,0.0,8.131721,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
5,1611053645875,1611053792125,52.0,35.338254,139.500717,gnss,0.0,10.151713,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
6,1611053585771,1611053792125,53.0,35.338249,139.500709,gnss,0.0,9.975325,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
7,1611053525749,1611053792125,52.0,35.338259,139.500700,gnss,0.0,11.854683,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
8,1611053465660,1611053492125,51.0,35.338268,139.500688,gnss,0.0,11.932480,1.611053e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
9,1611053405559,1611053492125,51.0,35.338274,139.500691,gnss,0.0,21.947155,1.611053e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN


In [20]:
#dfの緯度経度どちらかにnanが１つでもある行を削除
df2= df.dropna(subset=['position.lat', 'position.lng'])
print(len(df2))
df2

148


,timestamp,serverTimestamp,position.alt,position.lat,position.lng,position.type,position.speed,position.accuracy,position.timestamp,position.altaccuracy,system.client.name,system.client.model,system.client.version,system.client.firmware,system.client.platform,system.client.manufacturer,system.reportedSensorData.batteryLevel,system.reportedSensorData.batteryIsCharging,position.heading
0,1611053946054,1611054100975,49.0,35.338304,139.500769,gnss,0.0,16.150562,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
1,1611053886049,1611054100975,49.0,35.338303,139.500766,gnss,0.0,6.694888,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
2,1611053825947,1611054100975,49.0,35.338294,139.500772,gnss,0.0,7.703820,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
3,1611053765918,1611053792125,52.0,35.338265,139.500737,gnss,0.0,10.166524,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
4,1611053705895,1611053792125,53.0,35.338253,139.500722,gnss,0.0,8.131721,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
5,1611053645875,1611053792125,52.0,35.338254,139.500717,gnss,0.0,10.151713,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
6,1611053585771,1611053792125,53.0,35.338249,139.500709,gnss,0.0,9.975325,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
7,1611053525749,1611053792125,52.0,35.338259,139.500700,gnss,0.0,11.854683,1.611054e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
8,1611053465660,1611053492125,51.0,35.338268,139.500688,gnss,0.0,11.932480,1.611053e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN
9,1611053405559,1611053492125,51.0,35.338274,139.500691,gnss,0.0,21.947155,1.611053e+12,1.0,HERE Tracker,Pixel 3a,1.1.5,RQ1A.210105.002,android,Google,37,False,NaN


## Plot on map

In [21]:
a = df2.iloc[:, 3:5].values
print(a)
print(len(a))

[[ 35.33830382 139.50076906]
 [ 35.33830283 139.50076558]
 [ 35.33829415 139.50077231]
 [ 35.33826498 139.50073697]
 [ 35.33825346 139.50072245]
 [ 35.33825408 139.50071666]
 [ 35.33824909 139.50070919]
 [ 35.33825934 139.50070044]
 [ 35.33826772 139.50068763]
 [ 35.3382739  139.50069065]
 [ 35.3382342  139.5006209 ]
 [ 35.33828364 139.50069071]
 [ 35.33828721 139.50069256]
 [ 35.3382893  139.50070629]
 [ 35.33829708 139.50075313]
 [ 35.338301   139.50076329]
 [ 35.33829898 139.50078262]
 [ 35.33830426 139.50078106]
 [ 35.33830742 139.50078375]
 [ 35.3383095  139.50079755]
 [ 35.33831769 139.50080663]
 [ 35.3382389  139.5006288 ]
 [ 35.3382736  139.5006193 ]
 [ 35.33853997 139.50108858]
 [ 35.3383665  139.50065896]
 [ 35.33824491 139.50085557]
 [ 35.33819186 139.50094361]
 [ 35.33963864 139.49866784]
 [ 35.34027917 139.49864256]
 [ 35.34028769 139.49864509]
 [ 35.34029617 139.49865026]
 [ 35.34029454 139.49862883]
 [ 35.34023312 139.49855135]
 [ 35.34051308 139.49604915]
 [ 35.3403375 

In [22]:
# 中心点を設定（藤沢駅）
lat = 35.338528
lng = 139.487632
# 描画
m = folium.Map(location=[lat, lng], zoom_start=14)

In [23]:
# 線の地図レイヤへの追加
line = folium.vector_layers.PolyLine(
            locations= a,        
            color='blue',                                  
            weight=2)
m.add_child(line)

## Route matching

In [24]:
# nullを含む行を削除（要改善）
array = trace.get('data')[:70] + trace.get('data')[71:]
len(array)

148

In [25]:
positions = [[datetime.datetime.fromtimestamp(data['timestamp'] / 1000), data['position']] for data in array]
rows = [[dt.isoformat(), p['lat'], p['lng'], p.get('speed', ''), p.get('heading', '')] for dt, p in positions]
rows = [['TIMESTAMP', 'LATITUDE', 'LONGITUDE', 'SPEED_MPS', 'HEADING']] + rows
trace_csv = '\n'.join(map(lambda x: ','.join(map(str, x)), rows))
print(trace_csv)

TIMESTAMP,LATITUDE,LONGITUDE,SPEED_MPS,HEADING
2021-01-19T19:59:06.054000,35.33830382,139.50076906,0,
2021-01-19T19:58:06.049000,35.33830283,139.50076558,0,
2021-01-19T19:57:05.947000,35.33829415,139.50077231,0,
2021-01-19T19:56:05.918000,35.33826498,139.50073697,0,
2021-01-19T19:55:05.895000,35.33825346,139.50072245,0,
2021-01-19T19:54:05.875000,35.33825408,139.50071666,0,
2021-01-19T19:53:05.771000,35.33824909,139.50070919,0,
2021-01-19T19:52:05.749000,35.33825934,139.50070044,0,
2021-01-19T19:51:05.660000,35.33826772,139.50068763,0,
2021-01-19T19:50:05.559000,35.3382739,139.50069065,0,
2021-01-19T19:49:05.488000,35.3382342,139.5006209,,
2021-01-19T19:48:05.443000,35.33828364,139.50069071,0,
2021-01-19T19:47:05.343000,35.33828721,139.50069256,0,
2021-01-19T19:46:05.321000,35.3382893,139.50070629,0,
2021-01-19T19:45:05.299000,35.33829708,139.50075313,0,
2021-01-19T19:44:05.198000,35.338301,139.50076329,0,
2021-01-19T19:43:05.095000,35.33829898,139.50078262,0,
2021-01-19T19:42:05.08700

In [26]:
mode = 'fastest;car;traffic:enabled'
response = match_route(APIKEY, mode, trace_csv, region='JPN')
response

https://fleet.ls.hereapi.com/2/calculateroute.json?routeMatch=1&apiKey=_5dAuxQ82T1V-yVw9jhVVjNn3rkbctA5_E4PVnzmy1w&mode=fastest%3Bcar%3Btraffic%3Aenabled&region=JPN


{'response': {'route': [{'mode': {'type': 'matched',
     'transportModes': ['car'],
     'trafficMode': 'enabled'},
    'boatFerry': False,
    'railFerry': False,
    'waypoint': [{'linkId': '+17641467904',
      'mappedPosition': {'latitude': 35.33829, 'longitude': 139.50073},
      'originalPosition': {'latitude': 35.33830382, 'longitude': 139.50076906},
      'spot': 0.86143,
      'confidenceValue': 1.0,
      'elevation': 0.0,
      'headingDegreeNorthClockwise': 10000.0,
      'headingMatched': -25.0,
      'matchDistance': 4.75,
      'minError': 4.0,
      'routeLinkSeqNrMatched': 0,
      'speedMps': 0.0,
      'timestamp': 1611086346054,
      'breakDetected': True,
      'breakDuration': -600566},
     {'linkId': '+17641467904',
      'mappedPosition': {'latitude': 35.33829, 'longitude': 139.50073},
      'originalPosition': {'latitude': 35.33830283, 'longitude': 139.50076558},
      'spot': 0.86143,
      'confidenceValue': 1.0,
      'elevation': 0.0,
      'headingDegre

In [27]:
response['response']['route'][0]

{'mode': {'type': 'matched',
  'transportModes': ['car'],
  'trafficMode': 'enabled'},
 'boatFerry': False,
 'railFerry': False,
 'waypoint': [{'linkId': '+17641467904',
   'mappedPosition': {'latitude': 35.33829, 'longitude': 139.50073},
   'originalPosition': {'latitude': 35.33830382, 'longitude': 139.50076906},
   'spot': 0.86143,
   'confidenceValue': 1.0,
   'elevation': 0.0,
   'headingDegreeNorthClockwise': 10000.0,
   'headingMatched': -25.0,
   'matchDistance': 4.75,
   'minError': 4.0,
   'routeLinkSeqNrMatched': 0,
   'speedMps': 0.0,
   'timestamp': 1611086346054,
   'breakDetected': True,
   'breakDuration': -600566},
  {'linkId': '+17641467904',
   'mappedPosition': {'latitude': 35.33829, 'longitude': 139.50073},
   'originalPosition': {'latitude': 35.33830283, 'longitude': 139.50076558},
   'spot': 0.86143,
   'confidenceValue': 1.0,
   'elevation': 0.0,
   'headingDegreeNorthClockwise': 10000.0,
   'headingMatched': -25.0,
   'matchDistance': 4.35,
   'minError': 100000

In [28]:
df_m = pd.json_normalize(response['response']['route'][0]['waypoint'])
df_m

,linkId,spot,confidenceValue,elevation,headingDegreeNorthClockwise,headingMatched,matchDistance,minError,routeLinkSeqNrMatched,speedMps,timestamp,breakDetected,breakDuration,mappedPosition.latitude,mappedPosition.longitude,originalPosition.latitude,originalPosition.longitude
0,+17641467904,0.86143,1.00,0.0,10000.0,-25.0,4.75,4.0,0,0.0,1611086346054,True,-600566.0,35.33829,139.50073,35.338304,139.500769
1,+17641467904,0.86143,1.00,0.0,10000.0,-25.0,4.35,1000000.0,0,0.0,1611086286049,NaN,NaN,35.33829,139.50073,35.338303,139.500766
2,+17641467904,0.83717,1.00,0.0,10000.0,-25.0,4.97,5.0,0,0.0,1611086225947,NaN,NaN,35.33828,139.50073,35.338294,139.500772
3,+17641467904,0.80196,1.00,0.0,10000.0,-25.0,0.65,1.0,0,0.0,1611086165918,NaN,NaN,35.33827,139.50074,35.338265,139.500737
4,+17641467904,0.78852,1.00,0.0,10000.0,-25.0,2.08,2.0,0,0.0,1611086105895,NaN,NaN,35.33826,139.50074,35.338253,139.500722
5,+17641467904,0.78852,1.00,0.0,10000.0,-25.0,2.68,1000000.0,0,0.0,1611086045875,NaN,NaN,35.33826,139.50074,35.338254,139.500717
6,+17641467904,0.78994,1.00,0.0,10000.0,-25.0,3.64,4.0,0,0.0,1611085985771,NaN,NaN,35.33826,139.50074,35.338249,139.500709
7,+17641467904,0.82011,1.00,0.0,10000.0,-25.0,4.56,5.0,0,0.0,1611085925749,NaN,NaN,35.33827,139.50074,35.338259,139.500700
8,+17641467904,0.84953,1.00,0.0,10000.0,-25.0,4.91,5.0,0,0.0,1611085865660,NaN,NaN,35.33828,139.50073,35.338268,139.500688
9,+17641467904,0.84953,1.00,0.0,10000.0,-25.0,4.43,1000000.0,0,0.0,1611085805559,NaN,NaN,35.33828,139.50073,35.338274,139.500691


In [29]:
arr = df_m.iloc[:, 13:15].values
print(arr)

# 線の地図レイヤへの追加
line = folium.vector_layers.PolyLine(
            locations= arr,        
            color='red',                                  
            weight=2)
m.add_child(line)

[[ 35.33829 139.50073]
 [ 35.33829 139.50073]
 [ 35.33828 139.50073]
 [ 35.33827 139.50074]
 [ 35.33826 139.50074]
 [ 35.33826 139.50074]
 [ 35.33826 139.50074]
 [ 35.33827 139.50074]
 [ 35.33828 139.50073]
 [ 35.33828 139.50073]
 [ 35.33829 139.50058]
 [ 35.33832 139.50066]
 [ 35.33832 139.50066]
 [ 35.33833 139.50068]
 [ 35.33834 139.5007 ]
 [ 35.33834 139.5007 ]
 [ 35.33834 139.5007 ]
 [ 35.33834 139.5007 ]
 [ 35.33834 139.5007 ]
 [ 35.33834 139.5007 ]
 [ 35.33834 139.5007 ]
 [ 35.33829 139.50059]
 [ 35.3383  139.5006 ]
 [ 35.33834 139.5007 ]
 [ 35.33833 139.50068]
 [ 35.33834 139.5007 ]
 [ 35.33834 139.5007 ]
 [ 35.33964 139.49867]
 [ 35.34016 139.49835]
 [ 35.34016 139.49835]
 [ 35.34017 139.49834]
 [ 35.34017 139.49834]
 [ 35.34016 139.49835]
 [ 35.34048 139.49605]
 [ 35.34038 139.49259]
 [ 35.34012 139.49018]
 [ 35.34017 139.48656]
 [ 35.34124 139.48383]
 [ 35.34016 139.48255]
 [ 35.33927 139.48605]
 [ 35.33927 139.48606]
 [ 35.33938 139.48737]
 [ 35.33938 139.48737]
 [ 35.33938

In [30]:
#map matching前後の比較（blue：前/red：後）
m

## 駐車位置プロット（mapmatchあり）

In [31]:
print(arr.shape)
arr[:5]

(148, 2)


array([[ 35.33829, 139.50073],
       [ 35.33829, 139.50073],
       [ 35.33828, 139.50073],
       [ 35.33827, 139.50074],
       [ 35.33826, 139.50074]])

In [32]:
#空の配列を用意
array = []
# 駐車と見做す誤差の値を設定
x = 0.00002

for num in range(1, len(arr)):
# 以下for文
    margin_lat = abs(arr[num-1][0] - arr[num][0])
    print('margin_lat: ', margin_lat)
    margin_lng = abs(arr[num-1][1] - arr[num][1])
    print('margin_lng: ', margin_lng)
    # if文：「前の時刻との緯度差分 + 経度差分 <= 0.00002」であれば駐車していると見做す
    margin = margin_lat + margin_lng
    if margin <= x:
        print('駐車位置')
        array.append(num)
    else:
        print('移動中')

margin_lat:  0.0
margin_lng:  0.0
駐車位置
margin_lat:  1.0000000003174137e-05
margin_lng:  0.0
駐車位置
margin_lat:  9.999999996068709e-06
margin_lng:  1.0000000003174137e-05
駐車位置
margin_lat:  1.0000000003174137e-05
margin_lng:  0.0
駐車位置
margin_lat:  0.0
margin_lng:  0.0
駐車位置
margin_lat:  0.0
margin_lng:  0.0
駐車位置
margin_lat:  1.0000000003174137e-05
margin_lng:  0.0
駐車位置
margin_lat:  9.999999996068709e-06
margin_lng:  1.0000000003174137e-05
駐車位置
margin_lat:  0.0
margin_lng:  0.0
駐車位置
margin_lat:  1.0000000003174137e-05
margin_lng:  0.00014999999999076863
移動中
margin_lat:  3.0000000002416982e-05
margin_lng:  7.999999999697138e-05
移動中
margin_lat:  0.0
margin_lng:  0.0
駐車位置
margin_lat:  9.999999996068709e-06
margin_lng:  1.9999999977926564e-05
移動中
margin_lat:  1.0000000003174137e-05
margin_lng:  2.0000000006348273e-05
移動中
margin_lat:  0.0
margin_lng:  0.0
駐車位置
margin_lat:  0.0
margin_lng:  0.0
駐車位置
margin_lat:  0.0
margin_lng:  0.0
駐車位置
margin_lat:  0.0
margin_lng:  0.0
駐車位置
margin_lat:  0.0
marg

In [33]:
df_p = pd.DataFrame(a[array])
print(df_p.shape)
df_p.head()

(59, 2)


,0,1
0,35.338303,139.500766
1,35.338294,139.500772
2,35.338265,139.500737
3,35.338253,139.500722
4,35.338254,139.500717


## 駐車点を統一

In [34]:
# マスタを作成
stop_lat = [0]
stop_lng = [0]
y = 0.0008
count = 0

In [35]:
# 59行ある駐車位置候補場所から１つずつ取り出す
for p_lat, p_lng in zip(df_p[0].values, df_p[1].values):
    print("p_lat, p_lng", p_lat, p_lng)
    # 確定駐車位置から0個目を取り出す（初めは[0,0]）
    for s_lat, s_lng in zip(stop_lat, stop_lng):
        print("s_lat, s_lng:", s_lat, s_lng)
        # 駐車位置候補の0個目と、確定駐車位置の0個目の差分を取る
        margin_lat_after = abs(s_lat - p_lat)
        margin_lng_after = abs(s_lng - p_lng)
        margin_after = margin_lat_after + margin_lng_after
        # marginが一定以下（=同じ駐車位置）だったらカウントして、除外対象とする
        if margin_after  <= y:
            count = count + 1
            print('count:', count)
    #再度まで見た後、カウント数が0個だったら（=どの既存駐車位置にも当てはまらなかったら）記録
    if count == 0:
        stop_lat.append(p_lat)
        stop_lng.append(p_lng)
    else:
        print("重複、スキップ")
    #カウントをリセット
    count = 0

p_lat, p_lng 35.33830283 139.50076558
s_lat, s_lng: 0 0
p_lat, p_lng 35.33829415 139.50077231
s_lat, s_lng: 0 0
s_lat, s_lng: 35.33830283 139.50076558
count: 1
重複、スキップ
p_lat, p_lng 35.33826498 139.50073697
s_lat, s_lng: 0 0
s_lat, s_lng: 35.33830283 139.50076558
count: 1
重複、スキップ
p_lat, p_lng 35.33825346 139.50072245
s_lat, s_lng: 0 0
s_lat, s_lng: 35.33830283 139.50076558
count: 1
重複、スキップ
p_lat, p_lng 35.33825408 139.50071666
s_lat, s_lng: 0 0
s_lat, s_lng: 35.33830283 139.50076558
count: 1
重複、スキップ
p_lat, p_lng 35.33824909 139.50070919
s_lat, s_lng: 0 0
s_lat, s_lng: 35.33830283 139.50076558
count: 1
重複、スキップ
p_lat, p_lng 35.33825934 139.50070044
s_lat, s_lng: 0 0
s_lat, s_lng: 35.33830283 139.50076558
count: 1
重複、スキップ
p_lat, p_lng 35.33826772 139.50068763
s_lat, s_lng: 0 0
s_lat, s_lng: 35.33830283 139.50076558
count: 1
重複、スキップ
p_lat, p_lng 35.3382739 139.50069065
s_lat, s_lng: 0 0
s_lat, s_lng: 35.33830283 139.50076558
count: 1
重複、スキップ
p_lat, p_lng 35.33828721 139.50069256
s_lat, s_ln

In [36]:
df_s = pd.DataFrame({'lat': stop_lat[1:], 'lng': stop_lng[1:]})
print(df_s.shape)
df_s.head(3)

(8, 2)


,lat,lng
0,35.338303,139.500766
1,35.340288,139.498645
2,35.339385,139.486053


In [37]:
# マーカーの地図レイヤへの追加
for i, row in df_s.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup='駐車位置',
        icon=folium.Icon(color='blue')
    ).add_to(m)

In [38]:
m

## 渡すデータ
### 走行データ（mapmatch前）

In [39]:
# df
df_before = pd.DataFrame({'latitude': a[:,0], 'longitude': a[:,1]})
print(df_before.shape)
df_before.head(3)

(148, 2)


,latitude,longitude
0,35.338304,139.500769
1,35.338303,139.500766
2,35.338294,139.500772


In [40]:
# nparray
arr_before = a
print(arr_before.shape)
arr_before[:3]

(148, 2)


array([[ 35.33830382, 139.50076906],
       [ 35.33830283, 139.50076558],
       [ 35.33829415, 139.50077231]])

### 走行データ（mapmatch後）

In [41]:
# df
df_after = df_m.iloc[:, 13:15]
print(df_after.shape)
df_after.head(3)

(148, 2)


,mappedPosition.latitude,mappedPosition.longitude
0,35.33829,139.50073
1,35.33829,139.50073
2,35.33828,139.50073


In [42]:
# nparray
arr_after = df_m.iloc[:, 13:15].values
print(arr_after.shape)
arr_after[:3]

(148, 2)


array([[ 35.33829, 139.50073],
       [ 35.33829, 139.50073],
       [ 35.33828, 139.50073]])

### Stoppoint

In [43]:
# df
print(df_s.shape)
df_s.head(3)

(8, 2)


,lat,lng
0,35.338303,139.500766
1,35.340288,139.498645
2,35.339385,139.486053


In [44]:
# nparray
arr_s = df_s.values
print(arr_s.shape)
arr_s[:3]

(8, 2)


array([[ 35.33830283, 139.50076558],
       [ 35.34028769, 139.49864509],
       [ 35.33938469, 139.48605316]])